# Modélisation 

In [339]:
# Functions imports

import tqdm as notebook_tqdm

import importlib
import functions
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV

importlib.reload(functions)
from functions import *

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [340]:
x = pd.read_csv("../data/processed/x_processed.csv")  # 10605 rows × 10 columns
x_challenge = pd.read_csv("../data/processed/x_challenge_processed.csv")  # 4942 rows × 10 columns
x_challenge_unprocessed = pd.read_csv("../data/raw/X_test_GgyECq8.csv")
y = pd.read_csv("../data/raw/y_train_jJtXgMX.csv")  # 10605 rows × 2 columns

Préparation des données 

In [341]:
# y = y.drop(columns=['DELIVERY_START'], axis=1)
x = x.drop(columns=['DELIVERY_START'], axis=1)
x_challenge = x_challenge.drop(columns=['DELIVERY_START'], axis=1)


y_bin = pd.DataFrame({
    'spot_id_delta' : (y['spot_id_delta'] > 0).astype(int)
})

y_abs = pd.DataFrame({
    'spot_id_delta' : abs(y['spot_id_delta'])
})

# y_bin.head(10) # Quand c'est négatif c'est 0 et quand c'est positif c'est 1 (Signe à prévoir en classfication)
# y_abs.head(10) # La valeur absolue à prévoir en régression

In [342]:
# Normalisation 

scaler = StandardScaler()
x = scaler.fit_transform(x)

x = pd.DataFrame(x) # scaler renvoie des numpy.darray

In [343]:
forest_abs = RandomForestRegressor(
    n_estimators=400,
    min_samples_split=2, 
    min_samples_leaf=1,
    max_features='sqrt',
    max_depth=50,
    bootstrap=False,
    random_state=42
)

forest_bin = RandomForestClassifier( # La RSCV n'a pas amélioré les scores, peut-être la refaire
    n_estimators=200,
    min_samples_split=10, 
    min_samples_leaf=4,
    max_features='log2',
    max_depth=50,
    bootstrap=False,
    random_state=42
)

Cross validation pour le modèle de regression

In [344]:
# cross_validation(forest_abs, x, y_abs['spot_id_delta'], 5, 'neg_mean_absolute_percentage_error')

Cross validation pour le modèle de classification

In [345]:
cross_validation(forest_bin, x, y_bin['spot_id_delta'], 5, 'accuracy')

##### CROSS-VALIDATION SCORES ##### 

 accuracy  scores from cross-validation  [0.82508251 0.81659595 0.80433758 0.81612447 0.81046676] 
 accuracy  mean from cross-validation  0.8145214521452144 
 Standard deviation  0.006907400875742554 

 ###################################


In [346]:
# En gros c'est pas mal j'ai juste 9 en std donc je vaisfaire la RSCV pour améliorer ça surtout

# Optimisation

In [347]:
# On commence par une Randomized Search, on passera à un HyperOpt plus tard s'il faut car pour l'instant
# j'ai des problèmes avec l'importation d'un package

# Paramètres pour la RandomizedSearchCV

# param_dist = {
#     'n_estimators': [int(x) for x in np.linspace(start=100, stop=1200, num=12)],
#     'max_features': [None, 'sqrt', 'log2'],
#     'max_depth': [int(x) for x in np.linspace(10, 110, num=11)] + [None],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
#     'bootstrap': [True, False]
# }

# forest_random = RandomizedSearchCV (
#     estimator=forest_bin,
#     param_distributions=param_dist,
#     n_iter=100,
#     cv=3,
#     verbose=2,
#     n_jobs=-1
# )

# forest_random.fit(x, y_bin['spot_id_delta'])

# print(f"Meilleurs paramètres : {forest_random.best_params_}")

# Meilleurs paramètres pour le regressor : {'n_estimators': 400, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 50, 'bootstrap': False}



Prédiction

In [348]:
forest_abs.fit(x, y_abs['spot_id_delta'])
forest_bin.fit(x, y_bin['spot_id_delta'])

# metrics(forest, x_test, y_test, 'Métriques pour la première RandomForest', 'regressor')

RandomForestClassifier(bootstrap=False, max_depth=50, max_features='log2',
                       min_samples_leaf=4, min_samples_split=10,
                       n_estimators=200, random_state=42)

In [349]:
y_pred = pd.DataFrame({
    'DELIVERY_START' : x_challenge_unprocessed['DELIVERY_START']
    })

y_pred['spot_id_delta_abs'] = forest_abs.predict(x_challenge)
y_pred['spot_id_delta_bin'] = forest_bin.predict(x_challenge)


y_pred.head(5)

/Users/hugopuybareau/Documents/GitHub/ELMY_DATA_CHALLENGE/.venv/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/Users/hugopuybareau/Documents/GitHub/ELMY_DATA_CHALLENGE/.venv/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


,DELIVERY_START,spot_id_delta_abs,spot_id_delta_bin
0,2023-04-02 00:00:00+02:00,41.694524,0
1,2023-04-02 01:00:00+02:00,36.679723,0
2,2023-04-02 02:00:00+02:00,36.314216,1
3,2023-04-02 03:00:00+02:00,36.194538,1
4,2023-04-02 04:00:00+02:00,35.662438,1


In [350]:
y_pred['spot_id_delta_bin'] = y_pred['spot_id_delta_bin'].apply(lambda x: 1 if x==1 else -1)

y_pred['spot_id_delta'] = y_pred['spot_id_delta_abs'] * y_pred['spot_id_delta_bin']
y_pred = y_pred.drop(columns=['spot_id_delta_abs', 'spot_id_delta_bin'])

y_pred.head(5)

,DELIVERY_START,spot_id_delta
0,2023-04-02 00:00:00+02:00,-41.694524
1,2023-04-02 01:00:00+02:00,-36.679723
2,2023-04-02 02:00:00+02:00,36.314216
3,2023-04-02 03:00:00+02:00,36.194538
4,2023-04-02 04:00:00+02:00,35.662438


In [351]:
y_pred.to_csv("../data/predicted/y_pred_2.csv", index=False)